In [1]:
import pandas as pd
import numpy as np
import re
import difflib
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection, naive_bayes, svm
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
import warnings
warnings.filterwarnings("ignore")
import en_core_web_sm
import spacy
from scipy.spatial.distance import cosine
nlp = en_core_web_sm.load()
nlp_md = spacy.load("en_core_web_md")
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import gensim
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
import logging
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS

In [2]:
### We are building a function for collocation N-gram (1,3)

In [3]:
### write all functions required to do so

In [4]:
#read the model_data 
data = pd.read_csv('data_label.csv',index_col = 0)

In [5]:
stop = set(STOPWORDS)
#Function to remove stopwords
def remove_stopwords(data,col):
    ''' This function removes stopwords for all the columns provided as data and column as inputs'''
    data = data
    col = col
    new_list = []
    a = data[col]
    for i in range(0,len(a)):
        words = word_tokenize(a[i])
        res_words = []
        for word in words:
            if word not in stop:
                res_words.append(word)
            sentence = " ".join(res_words)
        new_list.append(sentence)
    return new_list

In [6]:
def reg_clean(data,col):
    ''' This function cleans the column for a given data and column'''
    new_list = []
    for i in range(0,len(data)):
        #special characters 
        a = re.sub(r'[^ a-zA-Z0-9]','',data.loc[i,col])
        #remove multiple spaces by a single space
        a = re.sub(r'\s+',' ',a)
        #timestamp
        a = re.sub(r'\b[0-9]{1,}am|[0-9]{1,}pm|[0-9]{4,}|[0-9]ish|1st|2nd|3rd|[0-9]{1,2}th|31st|[0-9]{1,}min(?:utes)?s?|[0-9]{1,}h(?:ou)?rs?|[0-9]{3,}\b','timestamp',a)
        a = re.sub(r'\b[0-9]{1,}timestamp\b','timestamp',a)
        #any numbers as digit
        a = re.sub(r'\b\d{1,}\b','digit',a)
        #number followed by a variable
        a = re.sub(r'\b\d{1,}[a-z]{0,}[0-9]{0,}','varchar',a)
        #html codes
        a = re.sub(r'<.+?>','html',a)
        a = re.sub(r'https|www','html',a)
        new_list.append(a)
    return new_list

In [7]:
#lemmatize
def lemmatize_sentence(data,col):
    ''' This function lemmatizes the column for a given data and column'''
    new_list = []
    a = data[col] 
    for i in range(0,len(a)):
        words = word_tokenize(a[i])
        res_words = []
        for word in words:
            res_words.append(lemmatizer.lemmatize(word).strip(string.punctuation))
        sentence = " ".join(res_words)
        new_list.append(sentence)
    return new_list

In [8]:
def collocation_brand(data,col='brand'):
    ''' This is the collocation function written only for brand column'''
    data_col = data[col]
    new_list = []
    for i in range(0,len(data_col)):
        words = word_tokenize(data_col[i])
        res_word = []
        for word in words:
            res_word.append(word)
        sentence = "_".join(res_word)
        new_list.append(sentence)
    data[col] = new_list

In [9]:
def collocation_desc(data,col='description'):
    ''' This is the collocation function written only for description column'''
    col_list = [['dry', 'clean'],['high','waist'],['pointed','toe']]
    data_col = data[col]
    new_list = []
    clean_list = ['clean','waist','toe']
    for i in range(0,len(data_col)):
        words = word_tokenize(data_col[i])
        len_words = len(words)-1
        for i in range(0,len_words):
            bi_word = []
            j = i+1
            bi_word.append(words[i])
            bi_word.append(words[j])
            if(bi_word in col_list):
                sentence = "_".join(bi_word)
                words[i] = sentence
        for word in words:
            if word in clean_list:
                words.remove(word)
        res_word = []
        for word in words:
            res_word.append(word)
        sentence = " ".join(res_word)
        new_list.append(sentence)
    data[col] = new_list

In [10]:
def collocation_brandcat(data,col='brand_category'):
    ''' This is a collacation function written just for brand category'''
    col_list = [['mid', 'heel'],['boot','ankle'],['shoe','pump'],['shoe','boot'],['shoulder','bag'],
                ['shoe','sandal'],['straight','leg'],['top','blouse'],['wide','leg'],['high','heel']]
    data_col = data[col]
    new_list = []
    clean_list = ['heel','ankle','pump','bag','sandal','leg','blouse']
    for i in range(0,len(data_col)):
        words = word_tokenize(data_col[i])
        len_words = len(words)-1
        for i in range(0,len_words):
            bi_word = []
            j = i+1
            bi_word.append(words[i])
            bi_word.append(words[j])
            if(bi_word in col_list):
                sentence = "_".join(bi_word)
                words[i] = sentence
        for word in words:
            if word in clean_list:
                words.remove(word)
        res_word = []
        for word in words:
            res_word.append(word)
        sentence = " ".join(res_word)
        new_list.append(sentence)
    data[col] = new_list

In [11]:
def collocation_name(data,col='name'):
    '''This collocation function is written just for name''' 
    col_list = [['high', 'waist'],['long', 'sleeve'],['skinny', 'jean'],['slide', 'sandal'],['calf', 'hair'],
                ['midi', 'dress'],['crossbody', 'bag'],['straight', 'leg'],['wide', 'leg'],['air', 'max'],
                ['shoulder', 'bag'],['leopard', 'print'],['pointy', 'toe'],['croc', 'embossed'],['midi', 'skirt'],
                ['cashmere', 'sweater'],['pima', 'cotton'],['mock', 'neck'],['ankle', 'skinny'],['silk', 'blouse'],
                ['maxi', 'dress'],['ankle', 'strap'],['ankle', 'boot']]
    clean_list = ['waist','sleeve','jean','sandal','hair','dress','bag','leg','max','bag','print','toe',
                  'embossed','skirt','sweater','cotton','neck','skinny','blouse','dress','strap','boot']
    data_col = data[col]
    new_list = []
    for i in range(0,len(data_col)):
        words = word_tokenize(data_col[i])
        len_words = len(words)-1
        for i in range(0,len_words):
            bi_word = []
            j = i+1
            bi_word.append(words[i])
            bi_word.append(words[j])
            if(bi_word in col_list):
                sentence = "_".join(bi_word)
                words[i] = sentence
        for word in words:
            if word in clean_list:
                words.remove(word)
        res_word = []
        for word in words:
            res_word.append(word)
        sentence = " ".join(res_word)
        new_list.append(sentence)
    data[col] = new_list

In [12]:
def collocation_details(data,col='details'):
    ''' This collocationfunction is written just for details column'''
    col_list = [['true', 'size'],['dry','clean'],['machine','wash'],['tumble','dry'],['hand','wash'],
                ['long','sleeve'],['high','rise']]
    data_col = data[col]
    new_list = []
    clean_list = ['size','clean','wash','dry','sleeve','rise']
    for i in range(0,len(data_col)):
        words = word_tokenize(data_col[i])
        len_words = len(words)-1
        for i in range(0,len_words):
            bi_word = []
            j = i+1
            bi_word.append(words[i])
            bi_word.append(words[j])
            if(bi_word in col_list):
                sentence = "_".join(bi_word)
                words[i] = sentence
        for word in words:
            if word in clean_list:
                words.remove(word)
        res_word = []
        for word in words:
            res_word.append(word)
        sentence = " ".join(res_word)
        new_list.append(sentence)
    data[col] = new_list

In [13]:
#removing the last row to test for the function

data_label = data.loc[:3913,:]

In [14]:
#creating fake data to test the function
brand = data.loc[3914:3915,'brand']
description = data.loc[3914:3915,'description']
brand_cat = data.loc[3914:3915,'brand_category']
name = data.loc[3914:3915,'name']
details = data.loc[3914:3915,'details']

In [15]:
#best vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,3),token_pattern=r'\b[a-zA-Z]{3,}\b',stop_words="english",min_df = 10,max_df = 0.7,max_features = 300)

In [16]:
#best function parameters 
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')

In [17]:
#definig the function to predict for new input
def predict_style(brand,description,brand_cat,name,details,train_data = data_label):
    ''' This function has 6 inputs train_data with default input as data_label '''
    ''' Brand, description,brand_cat,name, and details have to be fed using a series'''
    len_ref = len(train_data)
    data = train_data[['brand','description','brand_category','name','details']]
    data_x = pd.DataFrame()
    print(len(data))
    data_x['brand'] = data['brand'].append(brand)
    data_x['description'] = data['description'].append(description)
    data_x['brand_category'] = data['brand_category'].append(brand_cat)
    data_x['name'] = data['name'].append(name)
    data_x['details'] = data['details'].append(details)
    data_x.reset_index(drop = True)
    print(len(data_x))
    print(data_x.shape)
    
    #First let's remove stopwords
    cols = data_x.columns
    for col in cols:
        remove_stopwords(data_x,col)
        
    #next, clean the data using regex 
    cols = data_x.columns
    for col in cols:
        data_label[col] = reg_clean(data_label,col)
    
    #next,lemmatize the data
    cols = data_x.columns
    for col in cols:
        data_label[col] = lemmatize_sentence(data_label,col)
    #then, run collocations of the data 
    collocation_brand(data_x)
    collocation_desc(data_x)
    collocation_brandcat(data_x)
    collocation_name(data_x)
    collocation_details(data_x)
    #Vectorize the entir corpus
    vect_data = pd.DataFrame()
    cols = data_x.columns
    for j in cols:
        corpus = []
        for i in range(0,len(data_x)):
            corpus.append(data_x.loc[i,j])
        vect = vectorizer.fit_transform(corpus)
        terms = vectorizer.get_feature_names()
        c=pd.DataFrame(vect.toarray().transpose(), index=terms)
        vect_data=pd.concat([vect_data,c.T],axis = 1)
    print(f'The Dimensionality of the vectorized data is:{vect_data.shape}')
    #now, let's build the model on length data
    X_train = vect_data[:len_ref]
    X_test = vect_data[len_ref:]
    #for is_casual
    Y_train = train_data['is_casual']
    SVM.fit(X_train,Y_train)
    predictions_SVM = SVM.predict(X_test)
    for i in range(0,len(predictions_SVM)):
        if predictions_SVM[i]:
            print(f'The product in document {i} "is casual"')
        else:
            print(f'The product in document {i} "is not casual"')    
    #for is_modern
    Y_train = train_data['is_modern']
    SVM.fit(X_train,Y_train)
    predictions_SVM = SVM.predict(X_test)
    for i in range(0,len(predictions_SVM)):
        if predictions_SVM[i]:
            print(f'The product in document {i} "is modern"')
        else:
            print(f'The product in document {i} "is not modern"')
    #for is_edgy
    Y_train = train_data['is_edgy']
    SVM.fit(X_train,Y_train)
    predictions_SVM = SVM.predict(X_test)
    for i in range(0,len(predictions_SVM)):
        if predictions_SVM[i]:
            print(f'The product in document {i} "is edgy"')
        else:
            print(f'The product in document {i} "is not edgy"')
        

In [18]:
#calling the functions
predict_style(brand,description,brand_cat,name,details)

3914
3916
(3916, 5)
The Dimensionality of the vectorized data is:(3916, 1016)
The product in document 0 "is not casual"
The product in document 1 "is casual"
The product in document 0 "is not modern"
The product in document 1 "is not modern"
The product in document 0 "is edgy"
The product in document 1 "is not edgy"


In [19]:
### Creating excel file for entire data

In [ ]:
#First let's remove stopwords
cols = data_x.columns
for col in cols:
    remove_stopwords(data_x,col)
        
    #next, clean the data using regex 
    cols = data_x.columns
    for col in cols:
        data_label[col] = reg_clean(data_label,col)
    
    #next,lemmatize the data
    cols = data_x.columns
    for col in cols:
        data_label[col] = lemmatize_sentence(data_label,col)
    #then, run collocations of the data 
    collocation_brand(data_x)
    collocation_desc(data_x)
    collocation_brandcat(data_x)
    collocation_name(data_x)
    collocation_details(data_x)
    #Vectorize the entir corpus
    vect_data = pd.DataFrame()
    cols = data_x.columns
    for j in cols:
        corpus = []
        for i in range(0,len(data_x)):
            corpus.append(data_x.loc[i,j])
        vect = vectorizer.fit_transform(corpus)
        terms = vectorizer.get_feature_names()
        c=pd.DataFrame(vect.toarray().transpose(), index=terms)
        vect_data=pd.concat([vect_data,c.T],axis = 1)
    print(f'The Dimensionality of the vectorized data is:{vect_data.shape}')